In [1]:
!pip install pamda

In [2]:
from google.colab import drive
drive.mount('fema-fl-data-SR_Branch')

Mounted at fema-fl-data-SR_Branch


In [3]:
!pip install /content/fema-fl-data-SR_Branch/MyDrive/GitHub/fema-fl-data-SR_Branch/fema_model

Processing ./fema-fl-data-SR_Branch/MyDrive/GitHub/fema-fl-data-SR_Branch/fema_model
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fema_model: filename=fema_model-0.1-py3-none-any.whl size=11298 sha256=10bb820fa3c15069fb5f7a63a0e90dbf37da1d8d9bde4d3b001f74e986f560e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-b2sfdysw/wheels/e3/fa/91/be2874dba7a6cd1488352befd13bfec88293e3c87af703ad73
  Created wheel for pamda: filename=pamda-0.0.10-py3-none-any.whl size=10791 sha256=517a045f994e9d7cb2d3f3346c3d5acaea1f1e81e5a72f68ec18d38cbeb2d738
  Stored in directory: /root/.cache/pip/wheels/5b/5d/e2/c7497121708bb3d7bab3de1914561fb78b09331b697fed59cc
Successfully built fema_model pamda
  Attempting uninstall: pamda
    Found existing installation: pamda 2.5.0
    Uninstalling pamda-2.5.0:
      Successfully uninstalled pamda-2.5.0


In [4]:
import sys
sys.path.append('/content/fema-fl-data-SR_Branch/MyDrive/GitHub/fema-fl-data-SR_Branch')
import sys
sys.path.append('/content/fema-fl-data-SR_Branch/MyDrive/GitHub/fema-fl-data-Dependency_Files')
import sys
sys.path.append('/content/fema-fl-data-SR_Branch/MyDrive/GitHub/fema-fl-data-main')

In [5]:
from pamda import pamda as p
from fema_model import Multi_Period_Terminal_System
import pandas as pd
import random
from tqdm import tqdm
import numpy as np
from scipy import stats
import requests
import json
from statsmodels.stats.weightstats import DescrStatsW

In [6]:
# first drop the "geo_areas.json" to the files
geo_areas = p.read_json('./geo_areas.json')

In [7]:
# Numeric experiments for single sample terminal with different structure parameters
# Bay interventions

# Fixed parameters

# Number of gates
n_g = [1,2,3]
# Number of bays
n_b = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
# Station distances (miles)
d = [25,50,100,150,200,300]
# Number of stations
S = 325
# Bay queue space
b_q = 3

# Time to empty fuel at station (mins)
r_s = 60
# Bounds of time to empty fuel at station (mins)
r_s_bound = 10
# Truck start time max (hrs)
a = 6
# Truck capacity (gallons)
c = 9000
# Truck stopping criteria (hr)
tau = 0.5

# Flexible parameters

# Gate check-in time (mins)
r_g = 7
# Bounds multiplier of gate check-in time (mins)
r_g_bound = 0.1
# Bay filling time (mins)
r_b = [15,17.5,20,22.5,25,27.5,30,32.5,35,37.5,40,42.5,45]
# Bounds multiplier of bay filling time (mins)
r_b_bound = 0.1
# Truck speed (mph)
v = 45
# Bounds multiplier of truck speed (mph)
v_bound = 0.1
# Hours of service (hrs)
h = 14
# Fleet size (trucks)
f = 50

# Iterations
itrn = 30

# Final outputs for all experiments
ops_df = pd.DataFrame(columns = ['Iteration', 'Gates', 'Bays', 'Station_Dist', 'Bay_Time', \
                                 'Flow', 'Trips_Per_Truck', 'Gate_Queue_Wait', 'Driving_Time'])

# Input kwargs
inputs = {
    'carry_over_demand_multiplier':0,
    'default_kwargs':{
        'demand_situation':'dem_scen_1',
        'demand_multiplier':1,
        'truck_multiplier':1,
        'station_algorithm':'max',
        'truck_kwargs':{
            'size':c,
            'speed':v,
            'speed_sigma':v*v_bound,
            'empty_rate':r_s,
            'empty_rate_sigma':r_s_bound,
            'open_time_min':0,
            'open_time_max':a,
            'close_time_after_open':h,
            'close_early_delta':tau
        },
        'station_kwargs':{
            'open_time':0,
            'close_time':24,
        },
        'terminal_kwargs':{
            'gate_kwargs':{
                'gate_rate':r_g,
                'gate_rate_sigma':r_g*r_g_bound,
                'open_time':0,
                'close_time':24,
                'extra_gates':0,
                'max_gate_queue':None, #Infinite
                'share_gate_queue_bool':True
            },
            'bay_kwargs':{
                'open_time':0,
                'close_time':24,
                # 'fill_rate':35,
                # 'fill_rate_sigma':35*r_b_bound,
                'extra_bays':0,
                'max_bay_queue':b_q,
                'share_bay_queue_bool':True
            }
        }
    }
}

# Create terminal system dictionary
terminal_system_data = {}
terminal_system_data['group1'] = {}
terminal_system_data['group1']['trucks_available'] = f
terminal_system_data['group1']['terminals'] = {}
terminal_system_data['group1']['terminals']['terminal1'] = {}
terminal_system_data['group1']['terminals']['terminal1']['fuel_types'] = 'a'

# Index of ops_df
q = 0

for i in tqdm(range(len(n_g))):
    terminal_system_data['group1']['terminals']['terminal1']['num_gates'] = n_g[i]
    for j in tqdm(range(len(n_b))):
        terminal_system_data['group1']['terminals']['terminal1']['num_bays'] = n_b[j]
        for k in range(len(d)):
            for l in range(itrn):
                # Create stations list
                terminal_system_data['group1']['stations'] = {}
                for m in range(S):
                    station_name = 's'+str(m)
                    terminal_system_data['group1']['stations'][station_name] = {}
                    terminal_system_data['group1']['stations'][station_name]['travel_distance'] = round(random.uniform(0.01,d[k]),3)
                    terminal_system_data['group1']['stations'][station_name]['geo_code'] = '1'
                    terminal_system_data['group1']['stations'][station_name]['demand_scenarios'] = {}
                    terminal_system_data['group1']['stations'][station_name]['demand_scenarios']['dem_scen_1'] = {}
                    terminal_system_data['group1']['stations'][station_name]['demand_scenarios']['dem_scen_1']['demand'] = c
                    terminal_system_data['group1']['stations'][station_name]['demand_scenarios']['dem_scen_1']['fuel_types'] = 'a'
                for n in range(len(r_b)):
                    inputs['default_kwargs']['terminal_kwargs']['bay_kwargs']['fill_rate'] = r_b[n]
                    inputs['default_kwargs']['terminal_kwargs']['bay_kwargs']['fill_rate_sigma'] = r_b[n]*r_b_bound
                    periods = [inputs]
                    model = Multi_Period_Terminal_System(
                            terminal_system_data = terminal_system_data,
                            geo_areas = geo_areas,
                            periods = periods
                            )
                    output = model.serialize(minify = True)
                    ops_df.loc[q,'Iteration'] = l
                    ops_df.loc[q,'Gates'] = n_g[i]
                    ops_df.loc[q,'Bays'] = n_b[j]
                    ops_df.loc[q,'Station_Dist'] = d[k]
                    ops_df.loc[q,'Bay_Time'] = r_b[n]
                    ops_df.loc[q,'Flow'] = round((output['0']['statistics']['sum_met_total_demand']),3)
                    ops_df.loc[q,'Trips_Per_Truck'] = round((output['0']['statistics']['truck_avg_deliveries_made']),3)
                    ops_df.loc[q,'Gate_Queue_Wait'] = round((output['0']['statistics']['trip_avg_in_gate_queue']),3)
                    ops_df.loc[q,'Driving_Time'] = round((output['0']['statistics']['truck_avg_to_station'] + output['0']['statistics']['truck_avg_to_terminal_group']),3)
                    q = q+1

ops_df['Gate_Time'] = r_g
ops_df['Truck_Speed'] = v
ops_df['Truck_HoS'] = h
ops_df['Fleet'] = f

ops_df.to_csv('2024-11-13_Bay_Num_Exp1_30iter_Results.csv')

100%|██████████| 3/3 [4:53:26<00:00, 5868.90s/it]
